In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import vdmlab as vdm

In [ ]:
def test_filter(spikes, edges, gaussian_std):
    dt = np.median(np.diff(edges))

    if gaussian_std is not None:
        n_points = 3 * gaussian_std * 2 / dt
        if n_points > len(edges):
            raise ValueError("gaussian_std is too large for these times")
        if n_points < 2:
            print('No gaussian filter applied. Check that gaussian_std > dt if filter desired.')
        gaussian_filter = signal.gaussian(n_points, gaussian_std/dt)
        gaussian_filter /= np.sum(gaussian_filter)

    counts = np.zeros((len(spikes), len(edges)-1))
    for idx, spiketrain in enumerate(spikes):
        counts[idx] = np.histogram(spiketrain.time, bins=edges)[0]
        if gaussian_std is not None and gaussian_std > dt:
            counts[idx] = np.convolve(counts[idx], gaussian_filter, mode='same')
    return counts

In [ ]:
dt = np.median(np.diff(edges))
n_points = 3 * gaussian_std * 2 / dt
gaussian_filter = signal.gaussian(n_points, gaussian_std/dt)
gaussian_filter /= np.sum(gaussian_filter)

In [ ]:
mwin = 1.
mstd = 0.002
mdt = 0.025
mwindow = mwin / mdt 
if mwindow % 2 == 0:
    mwindow += 1
print(mwindow)
mgaussstd = mstd / mdt
print(mgaussstd)
mfilter = signal.gaussian(mwindow, mgaussstd)
print(mfilter)

In [ ]:
plt.plot(mfilter)
plt.show()

In [ ]:
pdt = 0.025
pwin = 0.025
pstd = 0.005

# n_points = 3 * pstd * 2 / pdt
n_points = pwin / pdt
print(n_points)
gaussian_filter = signal.gaussian(n_points, pstd/pdt)
# gaussian_filter /= np.sum(gaussian_filter)

In [ ]:
gaussian_filter

In [ ]:
signal = np.random.uniform(-1, 1, size=100)
plt.plot(signal)
plt.plot(np.convolve(signal, gaussian_filter))

In [ ]:
sig = vdm.SpikeTrain(np.array([ 0., 0., 0., 0., 1., 1., 1., 2., 2., 2., 2.]))
spikes = [sig]
edges = np.array([0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [ ]:
spikes[0].time

In [ ]:
dt

In [ ]:
gaussian_std = 0.1
c = test_filter(spikes, edges, gaussian_std)

In [ ]:
c

In [ ]:
np.convolve([1,2,3], [0,1,0.5], 'same')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from load_data import get_pos, get_spikes, get_lfp
from tuning_curves_functions import get_tc_1d, get_odd_firing_idx

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3
import info.R063d4_info as r063d4
import info.R063d5_info as r063d5
import info.R063d6_info as r063d6
import info.R066d1_info as r066d1
import info.R066d2_info as r066d2
import info.R066d3_info as r066d3
import info.R066d4_info as r066d4
import info.R067d1_info as r067d1
import info.R067d2_info as r067d2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.signal

In [ ]:
times = np.linspace(0, 0.1, 11)

In [ ]:
times

In [ ]:
gaussian_std = 0.01
dt = np.median(np.diff(times))
n_points = 3 * gaussian_std * 2 / dt

In [ ]:
out = scipy.signal.gaussian(n_points, gaussian_std/dt)

In [ ]:
out

In [ ]:
plt.plot(times[:len(out)], out)
plt.show()

In [ ]:
import vdmlab as vdm
from load_data import get_pos, get_spikes, get_lfp

import sys
sys.path.append('E:\\code\\python-vdmlab\\projects\\emily_shortcut\\info')
import info.r063d2 as r063d2
import info.r063d3 as r063d3
import info.r063d4 as r063d4
import info.r063d5 as r063d5
import info.r063d6 as r063d6
import info.r066d1 as r066d1
import info.r066d2 as r066d2
import info.r066d4 as r066d4
import info.r066d5 as r066d5
import info.r066d6 as r066d6
import info.r067d1 as r067d1
import info.r067d2 as r067d2
import info.r067d3 as r067d3
import info.r067d4 as r067d4
import info.r067d5 as r067d5
import info.r067d6 as r067d6
import info.r068d1 as r068d1
import info.r068d2 as r068d2
import info.r068d3 as r068d3
import info.r068d4 as r068d4
import info.r068d5 as r068d5
import info.r068d6 as r068d6

In [ ]:
info = r063d2

In [ ]:
spikes4 = get_spikes(info.session + '-spike4.mat')
spikes = get_spikes(info.session + '-spike.mat')

In [ ]:
print('4+s:', len(spikes4))
print('5s:', len(spikes) - len(spikes4))


In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'

In [ ]:
info = r063d4

In [ ]:
print(info.session_id)
pos = get_pos(info.pos_mat, info.pxl_to_cm)
csc = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
import numpy as np
import vdmlab as vdm
import matplotlib.pyplot as plt

In [ ]:
times = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
data = np.array([0.0, 0.5, 1.0, 0.7, 1.7])

In [ ]:
pos = vdm.Position(data, times)
speed = pos.speed()
speed.data

In [ ]:
what = pos[1:].distance(pos[:-1])

In [ ]:
what

In [ ]:
np.diff(pos.time)

In [ ]:
if not np.all(np.diff(pos.time) == np.median(np.diff(pos.time))):
    print('failed')

In [ ]:
np.median(np.diff(pos.time))

In [ ]:
import numpy as np
import vdmlab as vdm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from load_data import get_pos, get_spikes, get_lfp
import info.R063d3_info as r063d3
import info.R063d2_info as r063d2
import info.R063d4_info as r063d4

In [ ]:
info = r063d4

In [ ]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)

In [ ]:
speed = pos.speed(t_smooth=0.2)

In [ ]:
1/np.median(np.diff(pos.time))

In [ ]:
speed.data = speed.data * np.median(np.diff(pos.time))

In [ ]:
plt.plot(speed.time, speed.data, 'b.')
plt.ylim(0, 5)
plt.show()

In [ ]:
plt.hist(speed.data, bins=100)
plt.xlim(0, 50)
plt.show()

In [ ]:
thresh = 25
run_idx = np.squeeze(speed.data) >= thresh
running = pos[run_idx]

In [ ]:
plt.plot(running.x, running.y, 'g.')
plt.show()

In [ ]:
plt.plot(running.time, running.x, 'r.')
plt.show()

In [ ]:
plt.plot(running.time, running.y, 'c.')
plt.show()